In [ ]:
%pip install nltk
%pip install contractions
%pip install stopwords
%pip install gensim

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   - -------------------------------------- 1.0/24.4 MB 7.1 MB/s eta 0:00:04
   -- ------------------------------------- 1.6/24.4 MB 5.2 MB/s eta 0:00:05
   --- ------------------------------------ 2.1/24.4 MB 3.9 MB/s eta 0:00:06
   ---- ----------------------------------- 2.9/24.4 MB 3.6 MB/s eta 0:00:07
   ------ --------------------------------- 3.7/24.4 MB 3.6 MB/s eta 0:00:06
   -------- ------------------------------- 5.2/24.4 MB 4.3 MB/s eta 0:00:05
   ----------- ---------------------------- 6.8/24.4 MB 4.8 MB/s eta 0:00:04
   ------------- -------------------------- 8.4/24.4 MB 5.1 MB/s eta 0:00:04
   ---------------- ----------------------- 10.0/2

In [178]:
import re

import contractions
import gensim
#from bs4 import BeautifulSoup

import numpy as np
np.set_printoptions(threshold=10000, suppress=True)
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, zero_one_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, f1_score

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Nam\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("./CEAS_08.csv/CEAS_08.csv")

In [3]:
df.head()

,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1


In [5]:
def replace_contractions(text):
    """Supprime les contractions dans le texte."""
    return contractions.fix(text)

def replace_urls(text):
    """Supprime les URLs dans le texte."""
    return re.sub(r'http\S+', '<link>', text)

def remove_punctuation(text):
    """Supprime la ponctuation dans le texte."""
    return re.sub(r'[^\w ]', '', text)

def remove_non_ascii(text):
    """Supprime les caractères non ascii du texte."""
    return re.sub(r'[^\x00-\x7F]+', '', text)

def denoise_text(text):
    text = replace_urls(text)
    text = replace_contractions(text)
    text = remove_punctuation(text)
    text = remove_non_ascii(text)
    text = text.lower()
    return text



In [6]:
def remove_stopwords(text):
    """Remove stop words from list of tokenized words"""
    stop_words = stopwords.words('english')
    return [word for word in text if word not in stop_words]

def stem_words(text):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    return [stemmer.stem(word) for word in text]

def lemmatize_verbs(text):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, pos='v') for word in text]

def normalize(text):
    text = remove_stopwords(text)
    return text

In [72]:
def preprocess_text(text):
    text = denoise_text(text)
    # print(text)
    text = nltk.word_tokenize(text)
    text = normalize(text)
    text = stem_words(text)
    return ' '.join(text)

In [73]:
df["clean_body"] = df["body"].apply(preprocess_text)

In [74]:
df["clean_body"], df["label"]

(0        buck troubl caus smal dimend soon overbecom lo...
 1                              upgrad sex pleas techn link
 2        dai top 10 cnncom top video story aug 1 2008 3...
 3        would anyon object remov list tld bas dead fou...
 4                   welcomefastshippingcustomersupportlink
                                ...                        
 39149    cnn alert custom alert alert nam custom alerth...
 39150    cnn alert custom alert alert nam custom alerts...
 39151    hello gre work slid show view apply wond would...
 39152    mail send com intuitcomserv 206154105168 169 1...
 39153    dear fellow pythonistasguido acceiv us sit dir...
 Name: clean_body, Length: 39154, dtype: object,
 0        1
 1        1
 2        1
 3        0
 4        1
         ..
 39149    1
 39150    1
 39151    0
 39152    0
 39153    0
 Name: label, Length: 39154, dtype: int64)

## Séparation des données

In [96]:
X_train, X_test, y_train, y_test = train_test_split(df["clean_body"], df["label"], test_size=0.2, random_state=42)

X_train.info()

<class 'pandas.core.series.Series'>
Index: 31323 entries, 10951 to 15795
Series name: clean_body
Non-Null Count  Dtype 
--------------  ----- 
31323 non-null  object
dtypes: object(1)
memory usage: 489.4+ KB


## Vectorisation TF-IDF

In [97]:
vectorizer = TfidfVectorizer(max_features=1000)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.fit_transform(X_test)

In [83]:
vectorizer.get_feature_names_out()

array(['0000', '0100', '0500', '0800', '10', '100', '11', '12', '125000',
       '13', '14', '15', '16', '17', '18', '19', '20', '2005', '2006',
       '2007', '2008', '21', '22', '23', '24', '25', '2516195added',
       '2520262added', '2577781added', '26', '28', '30', '30303', '358',
       '40', '54', '64', '911',
       '_______________________________________________', 'abl', 'absurd',
       'acceiv', 'access', 'accord', 'account', 'achiev', 'acp', 'act',
       'ad', 'addit', 'address', 'admin', 'adv', 'advert', 'afternoon',
       'ag', 'agr', 'agree', 'airlin', 'airport', 'al', 'alert',
       'algorithm', 'alia', 'allow', 'already', 'also', 'alt', 'alway',
       'am', 'amy', 'analys', 'and', 'angel', 'annount', 'anoth', 'answ',
       'anthony', 'anthrax', 'antibiot', 'anyon', 'anyth', 'ap', 'app',
       'appear', 'appl', 'apply', 'approach', 'apr', 'april', 'ar',
       'arch', 'architect', 'area', 'argu', 'around', 'arrest', 'art',
       'artic', 'ask', 'assign', 'assocy

In [98]:
model = GaussianNB()

In [99]:
X_train = X_train.toarray()
X_test = X_test.toarray()

In [100]:
model = model.fit(X_train, y_train)

In [102]:
y_pred = model.predict(X_test)

In [119]:
def score(y_test, y_pred):
    print ("Accuracy:", accuracy_score(y_test, y_pred), 
           "Recall:", recall_score(y_test, y_pred),
            "F1-score:", f1_score(y_test, y_pred))

In [ ]:
score(y_test, y_pred)

In [115]:
from sklearn.svm import SVC

In [116]:
clf = SVC(gamma='auto')

clf.fit(X_train, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'auto'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [117]:
y_pred = clf.predict(X_test)

In [120]:
score(y_test, y_pred)

Accuracy: 0.7357936406589197 Recall: 0.8827459110803962 F1-score: 0.7874242268570841


## Word2Vec

In [124]:
from gensim.models import Word2Vec

In [126]:
import multiprocessing
cores=multiprocessing.cpu_count()
cores

12

In [127]:
w2vec = Word2Vec(sentences=df["clean_body"], vector_size=100, window=5,min_count=2,workers=cores-1)

In [129]:
for i in range(100):
    w2vec.train(df["clean_body"],total_examples=len(df["clean_body"]),epochs=1)
    print(i, end=' ')


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [130]:
w2vec.save('./models/Word2vec_entraine.h5')

In [137]:


def word2vec_generator(texts,model,vector_size):
    dict_word2vec = {}
    for index, word_list in enumerate(texts):
        arr = np.array([0.0 for i in range(0, vector_size)])
        nb_word=0
        for word in word_list:
            try:
                arr += model[word]
                nb_word=nb_word+1
            except KeyError:
                continue
        if(len(word_list) == 0):
            dict_word2vec[index] = arr
        else:
            dict_word2vec[index] = arr / nb_word
    df_word2vec = pd.DataFrame(dict_word2vec).T
    return df_word2vec



In [139]:
word2vec = word2vec_generator(df["clean_body"], w2vec.wv, 100)

In [165]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(word2vec, df["label"], test_size=0.2, random_state=42)

In [163]:
from sklearn.naive_bayes import MultinomialNB

In [168]:
model = MultinomialNB()
model = model.fit(X_train, y_train)

In [170]:
y_pred = model.predict(X_test)

In [148]:
proba = model.predict_proba(X_test2)

In [155]:
len(proba)

7831

In [160]:
proba

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       ...,
       [0.00000003, 0.99999997],
       [1.        , 0.        ],
       [0.99999999, 0.00000001]], shape=(7831, 2))

In [161]:
y_test2

5657     0
2133     1
12114    1
13852    0
35886    1
        ..
8931     1
35892    0
5575     0
38054    0
14734    0
Name: label, Length: 7831, dtype: int64

In [162]:
y_pred

array([1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,

In [171]:
score(y_test, y_pred)

Accuracy: 0.6512578214787383 Recall: 0.5443446210550564 F1-score: 0.6337669304009655


In [172]:
test = "Delivery reminder: the package is too large for the mailbox. Please choose a redelivery option via https://901758.reception-livreur.com"

In [175]:
test_preprocessed = preprocess_text(test)

In [176]:
vec_test = vectorizer.transform([test])

In [177]:
model.predict(vec_test)

array([1])

## Save du modèle

In [179]:
pickle.dump(model, open("./models/multinomialnb.pkl", "wb"))